In [1]:
import numpy as np
import pandas as pd
import joblib
from pandas import DataFrame
import xlsxwriter

from sklearn.impute import KNNImputer

#Get base from IBIS Data to fit
data = pd.ExcelFile("KNN Imputation.xlsx") 
ct_sheet = data.sheet_names[0]
sa_sheet = data.sheet_names[1]
ctsa_sheet = data.sheet_names[2]

ct_data = data.parse(ct_sheet)
sa_data = data.parse(sa_sheet)
ctsa_data = data.parse(ctsa_sheet)


data_features_ct = ct_data.loc[:, ct_data.columns] 
data_features_ct = data_features_ct.drop(['ROI','1CT-11142','1CT-12142','2CT-11142','2CT-12142'], axis=1)
data_features_sa = sa_data.loc[:, sa_data.columns] 
data_features_sa = data_features_sa.drop(['ROI','1SA-11142','1SA-12142','2SA-11142','2SA-12142'], axis=1)
#'1CT-11142','1CT-12142','2CT-11142','2CT-12142' Already dropped
#'1SA-11142','1SA-12142','2SA-11142','2SA-12142' Already dropped
data_features_ctsa = ctsa_data.loc[:, ctsa_data.columns] 
data_features_ctsa = data_features_ctsa.drop(['ROI'], axis=1)

print(data_features_ct.shape)
print(data_features_sa.shape)
print(data_features_ctsa.shape)

(290, 296)
(290, 296)
(290, 592)


In [2]:
ct_imputer = KNNImputer(n_neighbors=2)
sa_imputer = KNNImputer(n_neighbors=2)
ctsa_imputer  = KNNImputer(n_neighbors=2)
#Fit to IBIS Data
ct_imputer.fit(data_features_ct)
sa_imputer.fit(data_features_sa)
ctsa_imputer.fit(data_features_ctsa)

KNNImputer(n_neighbors=2)

In [3]:
#Get the Gilmore data we want imputed to calculate MAE
#'1CT-11142','1CT-12142','2CT-11142','2CT-12142' Already dropped
#'1SA-11142','1SA-12142','2SA-11142','2SA-12142' Already dropped
gilmore_ct_sheet_1yToImpute2y = data.sheet_names[3]
gilmore_ct_sheet_2yToImpute1y = data.sheet_names[4]
gilmore_sa_sheet_1yToImpute2y = data.sheet_names[5]
gilmore_sa_sheet_2yToImpute1y = data.sheet_names[6]
#Seeing which one imputes better. All at once, or individually CT and SA
#'1CT-11142','1CT-12142','2CT-11142','2CT-12142' Already dropped
#'1SA-11142','1SA-12142','2SA-11142','2SA-12142' Already dropped
gilmore_sheet_1yToImpute2y = data.sheet_names[7]
gilmore_sheet_2yToImpute1y = data.sheet_names[8]

gilmore_ct1y_data = data.parse(gilmore_ct_sheet_1yToImpute2y)
gilmore_data_features_ct1y = gilmore_ct1y_data.loc[:, gilmore_ct1y_data.columns]
gilmore_data_features_ct1y = gilmore_data_features_ct1y.drop(['ROI'], axis=1)

gilmore_ct2y_data = data.parse(gilmore_ct_sheet_2yToImpute1y)
gilmore_data_features_ct2y = gilmore_ct2y_data.loc[:, gilmore_ct2y_data.columns]
gilmore_data_features_ct2y = gilmore_data_features_ct2y.drop(['ROI'], axis=1)

gilmore_sa1y_data = data.parse(gilmore_sa_sheet_1yToImpute2y)
gilmore_data_features_sa1y = gilmore_sa1y_data.loc[:, gilmore_sa1y_data.columns]
gilmore_data_features_sa1y = gilmore_data_features_sa1y.drop(['ROI'], axis=1)

gilmore_sa2y_data = data.parse(gilmore_sa_sheet_2yToImpute1y)
gilmore_data_features_sa2y = gilmore_sa2y_data.loc[:, gilmore_sa2y_data.columns]
gilmore_data_features_sa2y = gilmore_data_features_sa2y.drop(['ROI'], axis=1)

print("Individual data feature shapes")
print(gilmore_data_features_ct1y.shape)
print(gilmore_data_features_ct2y.shape)
print(gilmore_data_features_sa1y.shape)
print(gilmore_data_features_sa2y.shape)

#####All at once now
gilmore_1y_data = data.parse(gilmore_sheet_1yToImpute2y)
gilmore_data_features_1y = gilmore_1y_data.loc[:, gilmore_1y_data.columns]
gilmore_data_features_1y = gilmore_1y_data.drop(['ROI'], axis=1)

gilmore_2y_data = data.parse(gilmore_sheet_2yToImpute1y)
gilmore_data_features_2y = gilmore_2y_data.loc[:, gilmore_2y_data.columns]
gilmore_data_features_2y = gilmore_2y_data.drop(['ROI'], axis=1)

print("Combined data feature shapes")
print(gilmore_data_features_1y.shape)
print(gilmore_data_features_2y.shape)

Individual data feature shapes
(129, 296)
(129, 296)
(129, 296)
(129, 296)
Combined data feature shapes
(129, 592)
(129, 592)


In [4]:
#Predict Gilmore Data for MAE
imputed_ct2y = ct_imputer.transform(gilmore_data_features_ct1y)
imputed_ct1y = ct_imputer.transform(gilmore_data_features_ct2y)
imputed_sa2y = sa_imputer.transform(gilmore_data_features_sa1y)
imputed_sa1y = sa_imputer.transform(gilmore_data_features_sa2y)
print("Imputed Individual Shapes")
print(imputed_ct2y.shape)
print(imputed_ct1y.shape)
print(imputed_sa2y.shape)
print(imputed_sa1y.shape)
#Combined Gilmore Data for MAE Comparison
imputed_2y = ctsa_imputer.transform(gilmore_data_features_1y)
imputed_1y = ctsa_imputer.transform(gilmore_data_features_2y)
print("Imputed Combined Shapes")
print(imputed_2y.shape)
print(imputed_1y.shape)

#DFs for Individuals
df_ct_2y = pd.DataFrame(imputed_ct2y)
df_ct_1y = pd.DataFrame(imputed_ct1y)
df_sa_2y = pd.DataFrame(imputed_sa2y)
df_sa_1y = pd.DataFrame(imputed_sa1y)

df_ct_2y.to_excel("Imputed CT 2y KNN.xlsx", index=False)
df_ct_1y.to_excel("Imputed CT 1y KNN.xlsx", index=False)
df_sa_2y.to_excel("Imputed SA 2y KNN.xlsx", index=False)
df_sa_1y.to_excel("Imputed SA 1y KNN.xlsx", index=False)

#DFs for Combined
df_1y = pd.DataFrame(imputed_1y)
df_2y = pd.DataFrame(imputed_2y)

df_1y.to_excel("Imputed 1y KNN.xlsx", index=False)
df_2y.to_excel("Imputed 2y KNN.xlsx", index=False)

#From here calculate MAE to see which imputation method within KNN works better.
#Use this same method to impute the data for the downstream method below.

#MAE for imputed 1y ct -
#MAE for imputed 2y ct -
#MAE for imputed 1y sa -
#MAE for imputed 2y sa -

#MAE for imputed 1y -
#MAE for imputed 2y -

Imputed Individual Shapes
(129, 296)
(129, 296)
(129, 296)
(129, 296)
Imputed Combined Shapes
(129, 592)
(129, 592)


In [5]:
#Get the data we want imputed
interpolate_data = pd.ExcelFile("Data to be Interpolated.xlsx") 
ct1y = interpolate_data.sheet_names[1]
sa1y = interpolate_data.sheet_names[3]
ct2y = interpolate_data.sheet_names[5]
sa2y = interpolate_data.sheet_names[7]

ct1y_data = interpolate_data.parse(ct1y)
data_features_ct1y = ct1y_data.loc[:, ct1y_data.columns] 
data_features_ct1y = data_features_ct1y.drop(['ROI'], axis=1)

sa1y_data = interpolate_data.parse(sa1y)
data_features_sa1y = sa1y_data.loc[:, sa1y_data.columns] 
data_features_sa1y = data_features_sa1y.drop(['ROI'], axis=1)

ct2y_data = interpolate_data.parse(ct2y)
data_features_ct2y = ct2y_data.loc[:, ct2y_data.columns] 
data_features_ct2y = data_features_ct2y.drop(['ROI'], axis=1) 

sa2y_data = interpolate_data.parse(sa2y)
data_features_sa2y = sa2y_data.loc[:, sa2y_data.columns] 
data_features_sa2y = data_features_sa2y.drop(['ROI'], axis=1)

In [6]:
#Predict the data we want to impute here for the downstream task
predicted_ct1y = ???_imputer.transform(data_features_ct1y)
df_predicted_ct1y = pd.DataFrame(predicted_ct1y)
df.to_excel("Interpolated ct1y Downstream.xlsx", index=False)

predicted_sa1y = ???_imputer.transform(data_features_sa1y)
df_predicted_sa1y = pd.DataFrame(predicted_sa1y)
df.to_excel("Interpolated sa1y Downstream.xlsx", index=False)

predicted_ct2y = ???_imputer.transform(data_features_ct2y)
df_predicted_ct2y = pd.DataFrame(predicted_ct2y)
df.to_excel("Interpolated ct2y Downstream.xlsx", index=False)

predicted_sa2y = ???_imputer.transform(data_features_sa2y)
df_predicted_sa2y = pd.DataFrame(predicted_sa2y)
df.to_excel("Interpolated sa2y Downstream.xlsx", index=False)

SyntaxError: invalid syntax (<ipython-input-6-d915c9aade5d>, line 2)